<a href="https://colab.research.google.com/github/laurenf3395/GeospatialDataManiputlation/blob/main/Getting_to_know_Geospatial_Data_manipulation_and_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Map German states using Google Earth Engine

In [ ]:
# Import Earth Engine
import ee

# Trigger authentication flow
ee.Authenticate()

# Initialize Library
ee.Initialize(project="buildwithai-420817")

from PIL import Image
import folium
import geemap
import requests
import math


In [ ]:
germany_states = ee.FeatureCollection("FAO/GAUL/2015/level1") \
                  .filter(ee.Filter.eq("ADM0_NAME", "Germany"))

# Print metadata to confirm
print("Number of states in Germany:", germany_states.size().getInfo())
image = ee.Image().paint(germany_states, 0, 2)

# Get the centroid of Germany for centering the map
germany_centroid = germany_states.geometry().centroid().coordinates().getInfo()
lat, lon = germany_centroid[1], germany_centroid[0]

print(f"Map Center: Latitude = {lat}, Longitude = {lon}")


Number of states in Germany: 16
Map Center: Latitude = 51.05286656613144, Longitude = 10.372108688819564


In [ ]:
# Following tutorial from https://www.youtube.com/watch?v=E65mfjqIwZI
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr= 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

my_map = folium.Map(location=[51.0528, 10.3721], zoom_start=6)
my_map.add_ee_layer(image,{'palette':'FF0000'}, "FAO/GAUL/2015/level1")
my_map.add_child(folium.LayerControl())
display(my_map)



# Locate Frankfurt and Download a representative Image



In [ ]:
# Define the coordinates of Frankfurt am Main
frankfurt = ee.Geometry.Point([8.6821, 50.1109])  # Longitude, Latitude

# Center the map on Frankfurt and print confirmation
print("Frankfurt Coordinates: ", frankfurt.getInfo())

frankfurt_area_sqkm = 248.31
buffer_size = math.sqrt(frankfurt_area_sqkm/math.pi) #buffer size ~ 8.9 km

# Set the date range for imagery
start_date = '2023-01-01'
end_date = '2023-12-31'
frankfurt_bounds = frankfurt.buffer(buffer_size*1000)
# Load Sentinel-2 imagery of Frankfurt, filter by location and cloud cover
frankfurt_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
.filterBounds(frankfurt_bounds).filterDate(start_date, end_date)
.filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)).mean().clip(frankfurt_bounds))

# Select RGB bands (True Color) and clip to Frankfurt area
frankfurt_RGB = frankfurt_collection.select(['B4', 'B3', 'B2'])



Frankfurt Coordinates:  {'type': 'Point', 'coordinates': [8.6821, 50.1109]}


In [ ]:
# Create a Folium map centered at Frankfurt
m = geemap.Map(center=[50.1109, 8.6821], zoom=12)

# Add the Sentinel-2 image layer
vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
m.addLayer(frankfurt_RGB, vis_params, "Sentinel-2 Frankfurt")

# Display the map
m


Map(center=[50.1109, 8.6821], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

Downloading Frankfurt image as RBG PNG image for viewing

In [ ]:
frankfurt_image = ee.Image(frankfurt_RGB)
frankfurt_image_info = frankfurt_image.getInfo()

print("Image Info is a ", type(frankfurt_image_info))

print("Fields of Image Info:")
for key in frankfurt_image_info:
    print(key)

image_projection = frankfurt_image.projection()

# Define a region around Frankfurt using the image's projection
frankfurt_region = frankfurt.transform(image_projection, 1).buffer(buffer_size).bounds() # approximate size of frankfurt


frankfurt_url = frankfurt_image.getThumbURL(
    {
        "format": "png",
        "bands": ["B4", "B3", "B2"],
        "dimensions": [1024, 1024],
        "region": frankfurt_region,
        "min": 0,
        "max": 3000,
    }
)

frankfurt_thumbURL_response = requests.get(frankfurt_url, stream=True)
frankfurt_PIL = Image.open(frankfurt_thumbURL_response.raw)
frankfurt_PIL.save("Sentinel-2_frankfurt.png")

Image Info is a  <class 'dict'>
Fields of Image Info:
type
bands
properties


### Calculating spectral indices using TIFF image with spectral bands

To know what the bands mean and what it represents: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/bands/

More information about the spectral indices: https://eos.com/blog/vegetation-indices/

1. For the EVI, we need NIR(B8), Blue(B2) and Red(B4) bands.
$$EVI = 2.5 \cdot (\frac{B8-B4}{B8+6 \cdot B4-7.5 \cdot B2 +1})$$
 The range of values for EVI is -1 to 1, with healthy vegetation generally around 0.20 to 0.80.

2. NDVI \\
Negative values to 0 mean no green
leaves. Values close to 1 indicate the highest
possible density of green leaves.
$$NDVI = \frac {(NIR - RED)} {(NIR+ RED)} = \frac {(B8 - B4)} {(B8+ B4)}$$

3. RVI \\
$$RVI = \frac {NIR}{RED} = \frac{B8}{B4}$$

4. ARVI \\
$$\frac{NIR - ((2 * RED) - BLUE)}{NIR + ((2 * RED) - BLUE)}$$

5. MNDWI \\

$$MNDWI = \frac{GREEN - SWIR}{GREEN + SWIR} = \frac {B3 - B11}{B3 + B11}$$

Green = pixel values from the green band and
SWIR = pixel values from the short-wave infrared band

For reducing the image to these values, used https://gis.stackexchange.com/questions/360278/extracting-ndvi-values-from-image-using-google-earth-engine-python-api

In [ ]:
def calculate_spectral_indices(rep_image):
  """
    Calculates spectral indices in representative image
    EVI, NEVI, MNDWI, RVI, NDVI, ARVI

    EVI: Enhanced Vegetation Index is a spectral index used in remote \
    sensing to measure vegetation greenness, essentially indicating the health \
    and density of plant life in an area

    NDVI: Normalized Difference Vegetation Index is used to measure the health \
    and density of vegetation

    RVI: Ratio Vegetation Index represents the ratio between the \
    near-infrared (NIR) and red reflectance values of a pixel, indicating amount\
    of vegetation present in that area; a higher RVI value signifies more \
    vegetation cover

    ARVI: Atmospherically Resistant Vegetation Index is a vegetation-based \
    index that minimizes the effects of atmospheric scattering

    NEVI: Normalized Enhanced Vegetation Index used to assess the amount and  \
    health of vegetation by comparing the reflectance values of near-infrared \
    and red light bands from a satellite image, similar to the widely known \
    NDVI (Normalized Difference Vegetation Index), but with additional adjustments

    MNDWI: Modified Normalized Difference Water Index which is a spectral index \
    used in remote sensing to identify and map water bodies

  """


  # EVI
  scaled_image = rep_image.select(['B8', 'B4', 'B2']).divide(10000)

  evi = scaled_image.expression(
    '2.5 * ((NIR - RED) / (NIR + (6 * RED) - (7.5 * BLUE) + 1))', {
        'NIR': scaled_image.select('B8'),  # NIR Band
        'RED': scaled_image.select('B4'),  # Red Band
        'BLUE': scaled_image.select('B2')  # Blue Band
    }).rename('EVI')

  # NDVI
  ndvi = rep_image.expression(
     '(NIR - RED) / (NIR + RED)', {
         'NIR': rep_image.select('B8'),  # NIR Band
         'RED': rep_image.select('B4') # RED Band
     }).rename('NDVI')

  # RVI
  rvi = rep_image.expression(
    '(NIR / RED)', {
        'NIR': rep_image.select('B8'),  # NIR Band
        'RED': rep_image.select('B4')  # Red Band
    }).rename('RVI')

  # ARVI
  arvi = rep_image.expression(
    '(NIR - ((2 * RED) - BLUE)) / (NIR + ((2 * RED) - BLUE))', {
        'NIR': rep_image.select('B8'), # NIR Band
        'RED': rep_image.select('B4'), # Red Band
        'BLUE': rep_image.select('B2') # Blue Band
    }).rename('ARVI')

  # MNDWI
  mndwi = rep_image.expression(
     '(GREEN - SWIR) / (GREEN + SWIR)', {
         'GREEN': rep_image.select('B3'), # Green Band
         'SWIR': rep_image.select('B11') # SWIR
     }).rename('MNDWI')

  return evi, ndvi, rvi, arvi, mndwi


Calculating statistics for the spectral indices: mean, min and max of the images for all the pixels

In [ ]:
def calculate_statistics(spectral_index_image, area_of_interest, SI_name):
  """
    spectral_index_image: ee.Image instance of highlighted spectral index
    area_of_interest: ee.Geometry instance of area of interest
    SI_name: string name of spectral index
  """
  stats = spectral_index_image.reduceRegion(reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(), sharedInputs=True),
    geometry=area_of_interest,
    scale=10,
    maxPixels=1e13)

  mean_SI = stats.get(f"{SI_name}_mean").getInfo()
  min_SI = stats.get(f"{SI_name}_min").getInfo()
  max_SI = stats.get(f"{SI_name}_max").getInfo()

  print(f"Mean {SI_name}: {mean_SI}")
  print(f"Min {SI_name}: {min_SI}")
  print(f"Max {SI_name}: {max_SI}")

In [ ]:
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
frankfurt_collection = frankfurt_collection.select(bands)


evi, ndvi, rvi, arvi, mndwi = calculate_spectral_indices(frankfurt_collection)

spectral_indices = {"EVI": evi, "NDVI": ndvi, "RVI": rvi, "ARVI": arvi, "MNDWI": mndwi}

for SI_name, SI_image in spectral_indices.items():
  calculate_statistics(SI_image, frankfurt_bounds, SI_name)
  print("\n")


m = geemap.Map()
m.centerObject(frankfurt_collection, 10) # zoom level 10

# Visualization parameters for each index
vis_params = {
    "EVI": {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']},
    "NDVI": {'min': -1, 'max': 1, 'palette': ['brown', 'white', 'green']},
    "MNDWI": {'min': -1, 'max': 1, 'palette': ['blue', 'cyan', 'white']},
    "RVI": {'min': 0, 'max': 10, 'palette': ['white', 'yellow', 'red']},
    "ARVI": {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']},
}

for index_name, index_image in spectral_indices.items():
    m.addLayer(index_image, vis_params[index_name], index_name)
m.addLayerControl()
m


Mean EVI: 0.30083016643389454
Min EVI: -31.11410601976675
Max EVI: 50.88054187192219


Mean NDVI: 0.5391489058910905
Min NDVI: -0.45580296896086364
Max NDVI: 0.8933838452171559


Mean RVI: 4.692406167035072
Min RVI: 0.37381228273464656
Max RVI: 17.758883248730964


Mean ARVI: 0.4618854652611583
Min ARVI: -145.75949367088526
Max ARVI: 131.83428571428794


Mean MNDWI: -0.45389712434038315
Min MNDWI: -0.8692467748647523
Max MNDWI: 0.7173509387249846




Map(center=[50.11090912057002, 8.682100394393244], controls=(WidgetControl(options=['position', 'transparent_b…

###Comparing spectral indices between winter and summer

Repeating the above exercise to compare spectral indices between Winter months(January & February) and Summer Months (May & June)

In [ ]:
winter_start_date = '2023-01-01'
winter_end_date = '2023-02-28'

winter_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
.filterBounds(frankfurt_bounds).filterDate(winter_start_date, winter_end_date)
.filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)).mean())

summer_start_date = '2023-05-01'
summer_end_date = '2023-06-30'

summer_collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
.filterBounds(frankfurt_bounds).filterDate(summer_start_date, summer_end_date)
.filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)).mean())

w_evi, w_ndvi, w_rvi, w_arvi, w_mndwi = calculate_spectral_indices(winter_collection)

winter_SI = {"EVI": w_evi, "NDVI": w_ndvi, "RVI": w_rvi, "ARVI": w_arvi, "MNDWI": w_mndwi}
print("Winter spectral indices")
for SI_name, SI_image in winter_SI.items():
  calculate_statistics(SI_image, frankfurt_bounds, SI_name)
  print("\n")

s_evi, s_ndvi, s_rvi, s_arvi, s_mndwi = calculate_spectral_indices(summer_collection)

summer_SI = {"EVI": s_evi, "NDVI": s_ndvi, "RVI": s_rvi, "ARVI": s_arvi, "MNDWI": s_mndwi}
print("Summer spectral indices")
for SI_name, SI_image in summer_SI.items():
  calculate_statistics(SI_image, frankfurt_bounds, SI_name)
  print("\n")

Winter spectral indices
Mean EVI: 0.18766315585531582
Min EVI: -90.81521739130557
Max EVI: 32.18886462882091


Mean NDVI: 0.4262472955081237
Min NDVI: -1
Max NDVI: 1


Mean RVI: 3.09056347311503
Min RVI: 0
Max RVI: 1666


Mean ARVI: 0.3114806729912113
Min ARVI: -233
Max ARVI: 366.46478873239437


Mean MNDWI: -0.45289156034599604
Min MNDWI: -1
Max MNDWI: 0.9132686084142395


Summer spectral indices
Mean EVI: 0.4202058680522225
Min EVI: -30.308219178082307
Max EVI: 116.4999999999999


Mean NDVI: 0.5761990443264222
Min NDVI: -1
Max NDVI: 1


Mean RVI: 6.4698048100384415
Min RVI: 0
Max RVI: 89.46666666666667


Mean ARVI: 0.5316307039189815
Min ARVI: -33.19047619047619
Max ARVI: 213.66666666666666


Mean MNDWI: -0.42946251793678686
Min MNDWI: -1
Max MNDWI: 0.875435672071526




### Cluster the pixels of an image

In [ ]:
# Combine the spectral indices into a multi-band image
indices_image = evi.addBands(ndvi).addBands(mndwi).addBands(rvi).addBands(arvi)
n_clusters = 5

# Make the training dataset.
training = indices_image.sample(region=frankfurt_bounds, scale=10, numPixels=5000)

# Instantiate the KMeans clusterer
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the multi-band image
clusters = indices_image.cluster(clusterer).rename('clusters')

Map = geemap.Map()
Map.centerObject(frankfurt_bounds, 10)

Map.addLayer(indices_image, {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette': ['brown', 'green']}, 'NDVI')
Map.addLayer(clusters.randomVisualizer(), {}, 'K-Means Clusters')
Map.addLayer(frankfurt_bounds, {'color': 'red'}, 'AOI')

# Display the map
Map


Map(center=[50.11090912057002, 8.682100394393244], controls=(WidgetControl(options=['position', 'transparent_b…